<a href="https://colab.research.google.com/github/SouroMLhub/CNN_MNIST_DATASET/blob/master/CNN(Convolutional_Nural_Network(MNIST)))).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [0]:
#initializing input
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

#units in convolutional Layer
n_convlayer1 = 32
n_convlayer2 = 64
#stride size for convolutional Layer
stride_conv1 = 1
stride_conv2 = 1
#filter size for convolutional Layer
conv1_k = 5
conv2_k = 5
#pooling layer
max_pool1_k = 2
max_pool2_k = 2
#hidden layer and output layer
n_hidden = 1024
n_out = 10
#input to hidden layer(7*7*64)
input_size_to_hiddden=(input_width//(max_pool1_k*max_pool2_k) *  input_height//(max_pool1_k*max_pool2_k)) * n_convlayer2 

In [0]:
#initializing Weights
weights = {
    #weight for convolutinal layers
    "wc1": tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_convlayer1])),
    "wc2": tf.Variable(tf.random_normal([conv2_k, conv2_k, n_convlayer1, n_convlayer2])),
    #weight for hidden layer
    "wh1": tf.Variable(tf.random_normal([input_size_to_hiddden, n_hidden])),
    #weight fro output layer
    "wo":  tf.Variable(tf.random_normal([n_hidden, n_out]))
    
}

#initializing biases
biases = {
    #biases for convolutional layers
    "bc1": tf.Variable(tf.random_normal([n_convlayer1])),
    "bc2": tf.Variable(tf.random_normal([n_convlayer2])),
    #bias for hidden layer
    "bh1": tf.Variable(tf.random_normal([n_hidden ])),
    #bias for output layer
    "bo": tf.Variable(tf.random_normal([n_out]))
}

In [0]:
#forward propagation
def cnn(x, weights, biases, keep_prob):
  #reshaping the mnist image
  x = tf.reshape(x, shape = [-1, input_width, input_height, input_channels])
  #calling funtion for convlutional layer1
  convLayer1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
  #calling funtion for maxpooling layer1
  conv1Pool = maxpooling(convLayer1, max_pool1_k)
  
  #calling funtion for convlutional layer2
  convLayer2 = conv(conv1Pool, weights['wc2'], biases['bc2'], stride_conv2)
  #calling funtion for maxpooling layer2
  conv2Pool = maxpooling(convLayer2, max_pool2_k)
  
  #reshaping(input to the hidden layer)
  hidden_input = tf.reshape(conv2Pool, shape = [-1, input_size_to_hiddden])
  #passing the input to the hidden layer(Dense layer)
  hidden_ouput_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
  #applying activation function to the hidden layer
  hidden_ouput_before_dropout = tf.nn.relu(hidden_ouput_before_activation)
  #hiddenlayer output after dropout
  hidden_output = tf.nn.dropout(hidden_ouput_before_dropout, keep_prob)
  #output(no activation function)
  output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo']) 
  return output

In [0]:
#Convolutional function
def conv(x, weights, bias, strides = 1):
  #applying filter to the image
  out = tf.nn.conv2d(x, weights, padding = "SAME", strides = [1, strides, strides, 1])
  #adding biases
  out = tf.nn.bias_add(out, bias)
  #activation function
  out = tf.nn.relu(out)
  return out

#Maxpooling function
def maxpooling(x, k= 2):
  return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k ,1], strides = [1, k, k, 1])

In [0]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

In [0]:
#fidning cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits =pred, labels = y))

In [0]:
#applying optimizer(Optimization)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
#Code for running the optimization multiple times using Batch Gradient Descent
batch_size = 100
for i in range(25):
  num_batches =int(mnist.train.num_examples/batch_size)
  total_cost = 0
  for j in range(num_batches):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    c, _ = sess.run([cost, optimize], feed_dict={x:batch_x, y:batch_y, keep_prob : 0.8})
    total_cost += c 
  print(total_cost)

1144648.5656599998
34682.76546555187
18659.35139129281
13083.63093914128
9394.254910353018
8383.55611038208
7405.76298181629
6633.619544106334
6014.135699672891
4906.571453391856
5441.963262381191
4757.388761868138
4264.480624604739
3615.06663277971
3469.1398968335634
3312.3576333317897
2874.959185911621
2777.0060368684285
2773.5190906186886
2480.374115091441
2403.4897282464844
1938.55092473013
1769.738767819651
2330.473597402875
1623.4606935428915


In [0]:
#Code to find accuracy on test data
predictions = tf.argmax(pred, 1)
true_labels = tf.argmax(y, 1)
correct_predictions = tf.equal(predictions, true_labels)

predictions_eval,labels, correct_pred  = sess.run([predictions, true_labels, 
                                                   correct_predictions], 
                                                  feed_dict={x:mnist.train.images, y:mnist.train.labels, keep_prob = 1.0})
correct_pred.sum()

54888